In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from datasets import Dataset

In [3]:
import pandas as pd

In [25]:
data = pd.read_csv('./autodl-tmp/dataset/val.csv')
data.head()

,Unnamed: 0,custom_id,system,user1,assistant1,user2,assistant2,total_tokens,answer1,answer2,qwen7b,qwen7_result_label_base,qwen7_result_label_simple,qwen7_result_label_simple_4bit,qwen7_result_label_simple_4bit_temp.5,qwen7_result_label_simple_4bit_temp.3,_merge
0,0,request-1,"{'role': 'system', 'content': '你是一个语义分类器.'}","{'role': 'user', 'content': '#任务：以下是对卷烟产品的评论，按...","{'role': 'assistant', 'content': '""分类标签"": ""B负面...","{'role': 'user', 'content': '请再次确认你的答案是否正确，并且分...","{'role': 'assistant', 'content': '我确认我的答案是正确的，...",233,B负面,B负面,"""分类标签"": ""B负面"",\n""理由"": ""评论中的词汇'太难抽'、'又苦又涩'均带有消极...",B负面,B负面,B负面,B负面,B负面,left_only
1,1,request-2,"{'role': 'system', 'content': '你是一个语义分类器.'}","{'role': 'user', 'content': '#任务：以下是对卷烟产品的评论，按...","{'role': 'assistant', 'content': '""分类标签"": ""B负面...","{'role': 'user', 'content': '请再次确认你的答案是否正确，并且分...","{'role': 'assistant', 'content': '抱歉造成了混淆。根据您的...",312,B负面,B负面,"""分类标签"": ""B负面"",\n""理由"": ""评论中的'味道真心不好'和'性价比真的低'表达...",B负面,D讽刺,B负面,B负面,B负面,left_only
2,2,request-3,"{'role': 'system', 'content': '你是一个语义分类器.'}","{'role': 'user', 'content': '#任务：以下是对卷烟产品的评论，按...","{'role': 'assistant', 'content': '""分类标签"": ""B负面...","{'role': 'user', 'content': '请再次确认你的答案是否正确，并且分...","{'role': 'assistant', 'content': '对不起，我之前的回答没有...",282,B负面,B负面,"""分类标签"": ""B负面"",\n""理由"": ""评论中的\""没咖啡味，不好抽\""表达了对产品的...",B负面,B负面,B负面,B负面,B负面,left_only
3,3,request-4,"{'role': 'system', 'content': '你是一个语义分类器.'}","{'role': 'user', 'content': '#任务：以下是对卷烟产品的评论，按...","{'role': 'assistant', 'content': '""分类标签"": ""B负面...","{'role': 'user', 'content': '请再次确认你的答案是否正确，并且分...","{'role': 'assistant', 'content': '抱歉造成了混淆。根据您提...",289,B负面,B负面,"""分类标签"": ""B负面"",\n""理由"": ""评论中提到的产品体验是负面的。评论者表示试用后...",B负面,B负面,B负面,B负面,B负面,left_only
4,4,request-5,"{'role': 'system', 'content': '你是一个语义分类器.'}","{'role': 'user', 'content': '#任务：以下是对卷烟产品的评论，按...","{'role': 'assistant', 'content': '""分类标签"": ""B负面...","{'role': 'user', 'content': '请再次确认你的答案是否正确，并且分...","{'role': 'assistant', 'content': '抱歉造成了混淆。根据您提...",243,B负面,B负面,"""分类标签"": ""B负面"",\n""理由"": ""评论中的\""太TM难抽了\""表达出对卷烟产品的...",B负面,B负面,B负面,B负面,B负面,left_only


In [26]:
data_for_dataset = data[['system','user1','assistant1','answer2']]

In [27]:
data_for_dataset.columns = ['instruction','input','response_total','response']

In [28]:
def format(x):
    return eval(x)['content'].replace('\'\'\'','')

In [32]:
data_for_dataset['response_total'] = data_for_dataset['response_total'].map(format)

/tmp/ipykernel_1431/3008227810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_dataset['response_total'] = data_for_dataset['response_total'].map(format)


In [33]:
data_for_dataset

,instruction,input,response_total,response
0,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""B负面"",""理由"": ""评论中的词汇'太难抽'、'又苦又涩'表达了消费者对...",B负面
1,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""B负面"",""理由"": ""评论中明确指出产品的味道不好，性价比低，这些都是直...",B负面
2,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""B负面"",""理由"": ""评论中明确表达了对于产品的不满，'没咖啡味，不好抽...",B负面
3,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""B负面"",""理由"": ""评论中明确表示了对产品的不满，提到了‘毫无咖啡味’...",B负面
4,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""B负面"",""理由"": ""评论中的“太TM难抽了”表达了强烈的负面情绪，'T...",B负面
...,...,...,...,...
36864,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""A正面"",""理由"": ""评论者对于卷烟产品的描述带有明显的怀旧情感，提到了...",A正面
36865,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""A正面"",""理由"": ""评论中提到‘特别好抽’，并且将产品与同等价位的其他...",A正面
36866,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""A正面"",""理由"": ""评论中提到‘太想念这个味道了’和‘太棒了’，明显表...",A正面
36867,你是一个语义分类器.,#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，简要说明理由# 评论：revi...,"""分类标签"": ""B负面"",""理由"": ""评论中提到了‘比硬中华劲大’，这通常意味着香烟的口...",C中性


In [22]:
train_data = Dataset.from_pandas(data_for_dataset)

In [34]:
val_data = Dataset.from_pandas(data_for_dataset)

In [36]:
val_data.save_to_disk('./autodl-tmp/dataset/datasets_val')

Saving the dataset (0/1 shards):   0%|          | 0/36869 [00:00<?, ? examples/s]

In [40]:
Dataset.load_from_disk('./autodl-tmp/dataset/datasets_val')

Dataset({
    features: ['instruction', 'input', 'response_total', 'response'],
    num_rows: 36869
})